In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import importlib
import os

from transliteration import data, train, model_one

In [2]:
importlib.reload(data)
batch_size = 128
train_dataset = data.make_dataset('../data/tfrecord/train.tfrecord',
                                  batch_size=batch_size)
valid_dataset = data.make_dataset('../data/tfrecord/valid.tfrecord',
                                  batch_size=batch_size)
test_dataset = data.make_dataset('../data/tfrecord/test.tfrecord',
                                 batch_size=batch_size)

In [3]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [13]:
importlib.reload(model_one)
encoder_config = model_one.Config(lstm_size=60,
                                  embedding_size=60,
                                  attention_size=30,
                                  vocab_size=data.ENGLISH_VOCAB_SIZE)
decoder_config = model_one.Config(lstm_size=60,
                                  embedding_size=30,
                                  attention_size=30,
                                  vocab_size=data.KATAKANA_VOCAB_SIZE)
encoder = model_one.Encoder(encoder_config)
decoder = model_one.Decoder(decoder_config)
start_token=data.intern_katakana_char('<start>')

In [17]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
importlib.reload(train)
best_val_loss = None
for e in range(200):
    loss = train.run_one_epoch(train_dataset,
                               True,
                               input_key='en',
                               output_key='ja',
                               encoder=encoder,
                               decoder=decoder,
                               optimizer=optimizer,
                               loss_function=loss_function,
                               start_token=start_token)
    valid_loss = train.run_one_epoch(valid_dataset,
                                     False,
                                     input_key='en',
                                     output_key='ja',
                                     encoder=encoder,
                                     decoder=decoder,
                                     loss_function=loss_function,
                                     start_token=start_token)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(train.transliterate_single(input_str='derick',
                           intern_input_fun=data.intern_en_char,
                           deintern_output_fun=data.deintern_katakana_char,
                           encoder=encoder,
                           decoder=decoder))

In [35]:
checkpoint.restore(checkpoint_prefix + '-8')

In [40]:
importlib.reload(train)
importlib.reload(data)
train.transliterate_single(input_str='william',
                           intern_input_fun=data.intern_en_char,
                           deintern_output_fun=data.deintern_katakana_char,
                           encoder=encoder,
                           decoder=decoder)

['ウィリアムム']

In [42]:
data.intern_katakana_char('<end>')

98